In [1]:
import re
from langchain_community.llms import Ollama
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.document_loaders import TextLoader
from langchain.embeddings import HuggingFaceEmbeddings  # Ensure compatibility with your setup
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA

# Initialize LLM
llm = Ollama(model="llama3.1")

# Load the text document
document_path = 'text_doc.md'  # Replace with your document path
loader = TextLoader(document_path)
documents = loader.load()

# Create embeddings and vectorstore
embeddings = HuggingFaceEmbeddings()
vectorstore = FAISS.from_documents(documents, embeddings)

# Create a retriever
retriever = vectorstore.as_retriever()

# Create a RetrievalQA chain
retrieval_qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

# Create a prompt for classification
classification_prompt = PromptTemplate.from_template(
    """
    Given the user's question and the assistant's answer, determine whether the assistant's answer addresses the user's question, it is okay even if the answer is only partially correct, as long as it is not completely empty of any information, in such cases start your answer with yes, otherwise no.

    Question: {question}
    Answer: {answer}
    """
)

# Create an LLM chain for classification
classification_chain = LLMChain(
    llm=llm,
    prompt=classification_prompt
)

def classify_answer(question):
    try:
        # Get answer from RetrievalQA chain
        response = retrieval_qa_chain({"query": question})
        answer = response["result"]
        print('RetrievalQA answer:', answer)

        # Use the LLM to classify whether the answer contains the information
        classification_input = {
            "question": question,
            "answer": answer
        }
        classification_output = classification_chain.run(classification_input).strip().lower()
        print('Raw Classification Output:', classification_output)

        # Extract 'yes' or 'no' from the classification output
        if 'yes' in classification_output:
            classification_result = 'yes'
        elif 'no' in classification_output:
            classification_result = 'no'
        else:
            # Default to 'no' if unable to determine
            classification_result = 'no'

        print('Classification result:', classification_result)
        return classification_result
    except Exception as e:
        print(f"Error in RetrievalQA or classification chain: {e}")
        return 'no'

# Example usage
if __name__ == "__main__":
    # Input question
    question = input("Enter your question: ")

    # Classify the answer
    result = classify_answer(question)

    # Output the result
    print(f"Final Classification Result: {result}")


/opt/miniconda3/envs/llm/lib/python3.9/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/var/folders/fm/4ph5fzys1_11v_27_wq8pnbc0000gn/T/ipykernel_89616/204070638.py:19: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings()
/var/folders/fm/4ph5fzys1_11v_27_wq8pnbc0000gn/T/ipykernel_89616/204070638.py:19: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecate

RetrievalQA answer: Yoga is the hobby of Sarah, as stated in the context that she is "a 25-year-old yoga enthusiast".


/var/folders/fm/4ph5fzys1_11v_27_wq8pnbc0000gn/T/ipykernel_89616/204070638.py:61: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  classification_output = classification_chain.run(classification_input).strip().lower()


Raw Classification Output: based on the user's question and the assistant's answer, i would say that the answer addresses the user's question. the user asked for sarah's hobbies, and the assistant provided a specific hobby (yoga) that they claim is associated with sarah.

while the assistant's answer might not be entirely comprehensive or definitive (e.g., it doesn't mention any other possible hobbies sarah may have), it provides some relevant information that directly answers the user's question.
Classification result: no
Final Classification Result: no
